# Processing large datasets with Apache Spark and Amazon SageMaker

***This notebook run on `Data Science 3.0 - Python 3` kernel on a `ml.t3.large` instance***.

Amazon SageMaker Processing Jobs are used  to analyze data and evaluate machine learning models on Amazon SageMaker. With Processing, you can use a simplified, managed experience on SageMaker to run your data processing workloads, such as feature engineering, data validation, model evaluation, and model interpretation. You can also use the Amazon SageMaker Processing APIs during the experimentation phase and after the code is deployed in production to evaluate performance.

        


![](https://docs.aws.amazon.com/images/sagemaker/latest/dg/images/Processing-1.png)

The preceding diagram shows how Amazon SageMaker spins up a Processing job. Amazon SageMaker takes your script, copies your data from Amazon Simple Storage Service (Amazon S3), and then pulls a processing container. The processing container image can either be an Amazon SageMaker built-in image or a custom image that you provide. The underlying infrastructure for a Processing job is fully managed by Amazon SageMaker. Cluster resources are provisioned for the duration of your job, and cleaned up when a job completes. The output of the Processing job is stored in the Amazon S3 bucket you specified.

## Our workflow for processing large amounts of data with SageMaker

We can divide our workflow into two steps:
    
1. Work with a small subset of the data with Spark running in local model in a SageMaker Studio Notebook.

1. Once we are able to work with the small subset of data we can provide the same code (as a Python script rather than a series of interactive steps) to SageMaker Processing which launched a Spark cluster, runs out code and terminates the cluster.

## In this notebook...

We will analyze the [Pushshift Reddit dataset](https://arxiv.org/pdf/2001.08435.pdf) to be used for the project and then we will run a SageMaker Processing Job to filter out the comments and submissions from subreddits of interest. The filtered data will be stored in your account's s3 bucket and it is this filtered data that you will be using for your project.

## Setup
We need an available Java installation to run pyspark. The easiest way to do this is to install JDK and set the proper paths using conda

In [5]:
# Setup - Run only once per Kernel App
%conda install openjdk -y

# install PySpark
%pip install pyspark==3.3.0

# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

Retrieving notices: ...working... done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 24.3.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.3.0



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - openjdk


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2024.3.11  |       h06a4308_0         127 KB
    certifi-2024.2.2           |  py310h06a4308_0         159 KB
    openjdk-11.0.13            |       h87a67e3_0       341.0 MB
    ------------------------------------------------------------
                                           Total:       341.3 MB

The following NEW packages will be INSTALLED:

  openjdk            pkgs/main/li

## Utilize S3 Data within local PySpark
* By specifying the `hadoop-aws` jar in our Spark config we're able to access S3 datasets using the s3a file prefix. 
* Since we've already authenticated ourself to SageMaker Studio , we can use our assumed SageMaker ExecutionRole for any S3 reads/writes by setting the credential provider as `ContainerCredentialsProvider`

In [6]:
# Import pyspark and build Spark session
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("PySparkApp")
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.2")
    .config(
        "fs.s3a.aws.credentials.provider",
        "com.amazonaws.auth.ContainerCredentialsProvider",
    )
    .getOrCreate()
)

print(spark.version)

:: loading settings :: url = jar:file:/opt/conda/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d2c4a961-d9fb-4601-9257-5a2a8e163fbb;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
:: resolution report :: resolve 422ms :: artifacts dl 38ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------------------

24/03/21 13:18:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


3.3.0


### Reading data into a Spark Dataframe
Note that we will be using the "s3a" adapter (read more [here](https://aws.amazon.com/blogs/opensource/community-collaboration-the-s3a-story)). S3A enables Hadoop to directly read and write Amazon S3 objects.

In [7]:
%%time
comments = spark.read.parquet(
    "s3a://bigdatateaching/reddit/parquet/comments/yyyy=*/mm=*/*comments*.parquet",
    header=True
)
comments.show()

24/03/21 13:18:39 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+-------------------+----------------------+--------------------+--------------------+----------------+-----------+-------------+------+------+-------+---------+----------+------------+-----+--------+--------------------+------------+
|             author|author_flair_css_class|   author_flair_text|                body|controversiality|created_utc|distinguished|edited|gilded|     id|  link_id| parent_id|retrieved_on|score|stickied|           subreddit|subreddit_id|
+-------------------+----------------------+--------------------+--------------------+----------------+-----------+-------------+------+------+-------+---------+----------+------------+-----+--------+--------------------+------------+
|          y26404986|                  null|                null|Mmmmm ... is this...|               0| 1641098735|         null|  null|     0|hqwcail|t3_rtzn1c| t3_rtzn1c|  1645567209|    1|   false|          RedditSets|   t5_3psukr|
|            iTwango|                  null|                

In [8]:
%%time
submissions = spark.read.parquet(
    "s3a://bigdatateaching/reddit/parquet/submissions/yyyy=*/mm=*/*submissions*.parquet",
    header=True
)
submissions.show()

+------------------+----------------------+--------------------+-----------+-------------+--------------------+-------------+------+-------+------+------------+-------+----------+------------+-----+--------------------+--------+--------------------+------------+--------------------+--------------------+
|            author|author_flair_css_class|   author_flair_text|created_utc|distinguished|              domain|       edited|    id|is_self|locked|num_comments|over_18|quarantine|retrieved_on|score|            selftext|stickied|           subreddit|subreddit_id|               title|                 url|
+------------------+----------------------+--------------------+-----------+-------------+--------------------+-------------+------+-------+------+------------+-------+----------+------------+-----+--------------------+--------+--------------------+------------+--------------------+--------------------+
|         [deleted]|                  null|                null| 1640997058|         

In [9]:
%%time
print(f"shape of the submissions dataframe is {submissions.count():,}x{len(submissions.columns)}") 

shape of the submissions dataframe is 313,972,841x21
CPU times: user 234 ms, sys: 56.2 ms, total: 290 ms
Wall time: 1min 35s


In [10]:
%%time
print(f"shape of the comments dataframe is {comments.count():,}x{len(comments.columns)}") 

shape of the comments dataframe is 3,076,907,105x17
CPU times: user 2.01 s, sys: 401 ms, total: 2.41 s
Wall time: 17min 29s


## Exploratory data analysis
Let us find the number of submissions and comments per subreddit. We will use SparkSQL for this.

In [11]:
submissions.createOrReplaceTempView("submissions")
comments.createOrReplaceTempView("comments")

In [12]:
%%time

sql_str="select subreddit, count(id) as count from submissions group by subreddit order by count desc"
submissions_subreddit_count = spark.sql(sql_str)
submissions_subreddit_count.show()

+-------------------+-------+
|          subreddit|  count|
+-------------------+-------+
|          AskReddit|2820312|
|       dirtykikpals|1797324|
|           dirtyr4r|1645658|
|        GaySnapchat|1294169|
|          jerkbudss|1226116|
|          teenagers| 992427|
|      DirtySnapchat| 924343|
|      FreeKarma4You| 740495|
|relationship_advice| 714845|
|      AutoNewspaper| 701442|
|        MassiveCock| 694869|
|            FemBoys| 653319|
|               cock| 642364|
|              memes| 631451|
|     wifepictrading| 621277|
|          Eldenring| 616378|
|           gonewild| 607159|
|         needysluts| 584244|
|     PokemonGoRaids| 575693|
|      Roleplaybuddy| 555313|
+-------------------+-------+
only showing top 20 rows

CPU times: user 454 ms, sys: 69.2 ms, total: 523 ms
Wall time: 6min 3s


In [25]:
import pyspark.sql.functions as f
submissions_subreddit_count.show(200)

+--------------------+-------+
|           subreddit|  count|
+--------------------+-------+
|           AskReddit|2820312|
|        dirtykikpals|1797324|
|            dirtyr4r|1645658|
|         GaySnapchat|1294169|
|           jerkbudss|1226116|
|           teenagers| 992427|
|       DirtySnapchat| 924343|
|       FreeKarma4You| 740495|
| relationship_advice| 714845|
|       AutoNewspaper| 701442|
|         MassiveCock| 694869|
|             FemBoys| 653319|
|                cock| 642364|
|               memes| 631451|
|      wifepictrading| 621277|
|           Eldenring| 616378|
|            gonewild| 607159|
|          needysluts| 584244|
|      PokemonGoRaids| 575693|
|       Roleplaybuddy| 555313|
|        dirtypenpals| 548776|
|    onlyfansgirls101| 532562|
|  HentaiAndRoleplayy| 531154|
|                Slut| 516687|
|       NSFW_Tributes| 507867|
|         WickrFinder| 506721|
|         FreeKarma4U| 505755|
|          ratemycock| 498468|
|               penis| 457594|
|       

## Process S3 data with SageMaker Processing Job `PySparkProcessor`

We are going to move the above processing code in a Python file and then submit that file to SageMaker Processing Job's [`PySparkProcessor`](https://sagemaker.readthedocs.io/en/stable/amazon_sagemaker_processing.html#pysparkprocessor).

In [13]:
!mkdir -p ./code

In [14]:
%%writefile ./code/process.py

import os
import logging
import argparse

# Import pyspark and build Spark session
from pyspark.sql.functions import *
from pyspark.sql.types import (
    DoubleType,
    IntegerType,
    StringType,
    StructField,
    StructType,
)
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

logging.basicConfig(format='%(asctime)s,%(levelname)s,%(module)s,%(filename)s,%(lineno)d,%(message)s', level=logging.DEBUG)
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))

def main():
    parser = argparse.ArgumentParser(description="app inputs and outputs")
    parser.add_argument("--s3_dataset_path_commments", type=str, help="Path of dataset in S3 for reddit comments")
    parser.add_argument("--s3_dataset_path_submissions", type=str, help="Path of dataset in S3 for reddit submissions")
    parser.add_argument("--s3_output_bucket", type=str, help="s3 output bucket")
    parser.add_argument("--s3_output_prefix", type=str, help="s3 output prefix")
    parser.add_argument("--subreddits", type=str, help="comma separate list of subreddits of interest")
    args = parser.parse_args()

    spark = SparkSession.builder.appName("PySparkApp").getOrCreate()
    logger.info(f"spark version = {spark.version}")
    
    # This is needed to save RDDs which is the only way to write nested Dataframes into CSV format
    sc = spark.sparkContext
    sc._jsc.hadoopConfiguration().set(
        "mapred.output.committer.class", "org.apache.hadoop.mapred.FileOutputCommitter"
    )

   
    # Downloading the data from S3 into a Dataframe
    logger.info(f"going to read {args.s3_dataset_path_commments}")
    comments = spark.read.parquet(args.s3_dataset_path_commments, header=True)
    logger.info(f"finished reading files...")
    
    logger.info(f"going to read {args.s3_dataset_path_submissions}")
    submissions = spark.read.parquet(args.s3_dataset_path_submissions, header=True)
    logger.info(f"finished reading files...")
    
    # filter the dataframe to only keep the subreddits of interest
    subreddits = [s.strip() for s in args.subreddits.split(",")]
    submissions_filtered = submissions.where(lower(col("subreddit")).isin(subreddits))
    comments_filtered = comments.where(lower(col("subreddit")).isin(subreddits))
    
    # save the filtered dataframes so that these files can now be used for future analysis
    s3_path = f"s3://{args.s3_output_bucket}/{args.s3_output_prefix}/comments"
    logger.info(f"going to write comments for {subreddits} in {s3_path}")
    logger.info(f"shape of the comments_filtered dataframe is {comments_filtered.count():,}x{len(comments_filtered.columns)}")
    comments_filtered.write.mode("overwrite").parquet(s3_path)
    
    s3_path = f"s3://{args.s3_output_bucket}/{args.s3_output_prefix}/submissions"
    logger.info(f"going to write submissions for {subreddits} in {s3_path}")
    logger.info(f"shape of the submissions_filtered dataframe is {submissions_filtered.count():,}x{len(submissions_filtered.columns)}")
    submissions_filtered.write.mode("overwrite").parquet(s3_path)

if __name__ == "__main__":
    main()

Writing ./code/process.py


Now submit this code to SageMaker Processing Job.

In [26]:
%%time
import sagemaker
from sagemaker.spark.processing import PySparkProcessor

# Setup the PySpark processor to run the job. Note the instance type and instance count parameters. SageMaker will create these many instances of this type for the spark job.
role = sagemaker.get_execution_role()
spark_processor = PySparkProcessor(
    base_job_name="sm-spark-project",
    framework_version="3.3",
    role=role,
    instance_count=6,
    instance_type="ml.m5.xlarge",
    max_runtime_in_seconds=3600,
)

# s3 paths
session = sagemaker.Session()
bucket = session.default_bucket()
s3_dataset_path_commments = "s3://bigdatateaching/reddit-parquet/comments/year=2021/month=1/*.parquet"
s3_dataset_path_submissions = "s3://bigdatateaching/reddit-parquet/submissions/year=2021/month=1/*.parquet"
output_prefix_data = "project"
output_prefix_logs = f"spark_logs"

# modify this comma separated list to choose the subreddits of interest
subreddits = "worldnews, AutoNewspaper. phnewsfeed, newsbotbot, news"
    
# run the job now, the arguments array is provided as command line to the Python script (Spark code in this case).
spark_processor.run(
    submit_app="./code/process.py",
    arguments=[
        "--s3_dataset_path_commments",
        s3_dataset_path_commments,
        "--s3_dataset_path_submissions",
        s3_dataset_path_submissions,
        "--s3_output_bucket",
        bucket,
        "--s3_output_prefix",
        output_prefix_data,
        "--subreddits",
        subreddits,
    ],
    spark_event_logs_s3_uri="s3://{}/{}/spark_event_logs".format(bucket, output_prefix_logs),
    logs=False,
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
..................................................................................................................!CPU times: user 1.85 s, sys: 191 ms, total: 2.05 s
Wall time: 9min 46s


## Read the filtered data

Now that we have filtered the data to only keep submissions and comments from subreddits of interest. Let us read data from the s3 path where we saved the filtered data.

In [27]:
%%time
s3_path = f"s3a://{bucket}/{output_prefix_data}/comments"
print(f"reading submissions from {s3_path}")
comments = spark.read.parquet(s3_path, header=True)
print(f"shape of the comments dataframe is {comments.count():,}x{len(comments.columns)}")

reading submissions from s3a://sagemaker-us-east-1-839279087569/project/comments


shape of the comments dataframe is 1,929,669x21
CPU times: user 23 ms, sys: 4.94 ms, total: 27.9 ms
Wall time: 34.3 s


In [28]:
comments.printSchema()

root
 |-- author: string (nullable = true)
 |-- author_cakeday: boolean (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- body: string (nullable = true)
 |-- can_gild: boolean (nullable = true)
 |-- controversiality: long (nullable = true)
 |-- created_utc: timestamp (nullable = true)
 |-- distinguished: string (nullable = true)
 |-- edited: string (nullable = true)
 |-- gilded: long (nullable = true)
 |-- id: string (nullable = true)
 |-- is_submitter: boolean (nullable = true)
 |-- link_id: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- permalink: string (nullable = true)
 |-- retrieved_on: timestamp (nullable = true)
 |-- score: long (nullable = true)
 |-- stickied: boolean (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- subreddit_id: string (nullable = true)



In [29]:
# display a subset of columns
comments.select("subreddit", "author", "body", "parent_id", "link_id", "id", "created_utc").show()

+---------+--------------------+--------------------+----------+---------+-------+-------------------+
|subreddit|              author|                body| parent_id|  link_id|     id|        created_utc|
+---------+--------------------+--------------------+----------+---------+-------+-------------------+
|     news|      workingmansalt|You don't need to...|t1_gilui37|t3_ktenem|gim53ll|2021-01-09 02:38:08|
|     news|          spazmodic-|He was coasting o...|t1_gim2w3a|t3_ktenem|gim53sd|2021-01-09 02:38:11|
|     news|International-Bit-36|Cue all the comme...| t3_ktenem|t3_ktenem|gim53u8|2021-01-09 02:38:12|
|worldnews|          HoopOnPoop|It is interesting...| t3_ktgri8|t3_ktgri8|gim53ue|2021-01-09 02:38:12|
|     news|           [deleted]|           [removed]|t1_gilq5e7|t3_ktenem|gim53uw|2021-01-09 02:38:12|
|worldnews|            strcrssd|No, we wouldn't. ...|t1_gil65ff|t3_kt90zj|gim53yf|2021-01-09 02:38:13|
|worldnews|  ZarathustraRiddled|Wait, he claimed ...|t1_gim47h7|t3_kt90zj

In [30]:
%%time
s3_path = f"s3a://{bucket}/{output_prefix_data}/submissions"
print(f"reading submissions from {s3_path}")
submissions = spark.read.parquet(s3_path, header=True)
print(f"shape of the submissions dataframe is {submissions.count():,}x{len(submissions.columns)}")

reading submissions from s3a://sagemaker-us-east-1-839279087569/project/submissions


shape of the submissions dataframe is 131,357x68
CPU times: user 15.6 ms, sys: 0 ns, total: 15.6 ms
Wall time: 14.9 s


In [31]:
submissions.printSchema()

root
 |-- adserver_click_url: string (nullable = true)
 |-- adserver_imp_pixel: string (nullable = true)
 |-- archived: boolean (nullable = true)
 |-- author: string (nullable = true)
 |-- author_cakeday: boolean (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- brand_safe: boolean (nullable = true)
 |-- contest_mode: boolean (nullable = true)
 |-- created_utc: timestamp (nullable = true)
 |-- crosspost_parent: string (nullable = true)
 |-- crosspost_parent_list: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- approved_at_utc: string (nullable = true)
 |    |    |-- approved_by: string (nullable = true)
 |    |    |-- archived: boolean (nullable = true)
 |    |    |-- author: string (nullable = true)
 |    |    |-- author_flair_css_class: string (nullable = true)
 |    |    |-- author_flair_text: string (nullable = true)
 |    |    

In [32]:
# display a subset of columns
submissions.select("subreddit", "author", "title", "selftext", "created_utc", "num_comments").show()

+---------+------------------+--------------------+--------+-------------------+------------+
|subreddit|            author|               title|selftext|        created_utc|num_comments|
+---------+------------------+--------------------+--------+-------------------+------------+
|     news|  popularnewsindia|When Shirtless Do...|        |2021-01-07 13:24:49|           0|
|worldnews|WoolfordStudiosLtd|Head Home Daily 0...|        |2021-01-07 13:25:12|           0|
|worldnews|         harryg888|Don’t drink alcoh...|        |2021-01-07 13:26:19|           0|
|worldnews|         harryg888|Are Your "Bridger...|        |2021-01-07 13:26:23|           0|
|worldnews|         harryg888|Trump promises or...|        |2021-01-07 13:26:27|           1|
|worldnews|         harryg888|Capitol Hill Riot...|        |2021-01-07 13:26:31|           0|
|worldnews|         harryg888|It's Time To Find...|        |2021-01-07 13:26:36|           0|
|worldnews|         harryg888|Dr Dre agrees to ...|        |